In [1]:
import httpx
import pandas as pd
import logging
import calendar
import unidecode
import openpyxl
import locale
from datetime import datetime, timedelta
import os

In [ ]:
# Definir a localização para português
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

In [2]:
# Configuração do logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%d/%m/%Y %H:%M:%S'
)

logger = logging.getLogger(__name__)

# Diminuir o nível de log para o httpx e outros loggers de terceiros
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

In [ ]:
# Diminuir o nível de log para o httpx e outros loggers de terceiros
logging.getLogger("httpx").setLevel(logging.WARNING)

# Dicionário de municípios não mapeados
ids_nao_mapeados = {
    "santana do livramento": 4317103,
    "são félix do araguaia": 5107859,
    "são luiz do paraitinga": 3550001,
    "mal. cândido rondon": 4114609,
    "mal. candido rondon": 4114609
}

Estação = "Convencional"
def preparar_nome_municipio(nome_municipio: str) -> str:
    """
    Prepara o nome do município para comparação:
    - Remove partes irrelevantes (após hífen ou parêntese)
    - Remove acentos
    - Converte para minúsculas
    - Remove espaços extras
    """
    nome_limpo = nome_municipio.split("-")[0].split("(")[0]
    return " ".join(unidecode.unidecode(nome_limpo).lower().split())

def fazer_requisicao(url: str, timeout: int = 10) -> list:
    """
    Faz a requisição para a API e retorna os dados.
    """
    try:
        response = httpx.get(url, timeout=timeout)
        response.raise_for_status()  # Levanta exceção para códigos HTTP >= 400
        dados = response.json()

        for item in dados:
            item["Estação"] = "Automatica"

        return dados
    except httpx.HTTPStatusError as e:
        logger.error(f"Erro HTTP {e.response.status_code} ao fazer requisição para {url}")
        return []
    except httpx.RequestError as e:
        logger.error(f"Erro na requisição: {e}")
        return []
    
def obter_codigo_ibge(nome_municipio: str, uf: str) -> dict:
    """
    Obtém o código IBGE do município. Caso não seja encontrado na base do IBGE,
    tenta buscar no dicionário de ids não mapeados. Retorna um dicionário com o código e o nome normalizado.
    """
    nome_preparado = preparar_nome_municipio(nome_municipio)
    logger.info(f"Nome preparado para comparação: '{nome_preparado}'")

    url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
    municipios = fazer_requisicao(url)

    if not municipios:
        logger.error(f"Erro ao obter dados do IBGE. Informe manualmente o código para {nome_municipio}.")
        return {"municipio_original": nome_municipio, "nome_comparacao": nome_preparado, "codigo_ibge": -1}

    # Tenta encontrar o município na lista do IBGE
    for municipio in municipios:
        nome_municipio_api = preparar_nome_municipio(municipio['nome'])
        uf_api = municipio['microrregiao']['mesorregiao']['UF']['sigla'].lower()
        logger.debug(f"Comparando '{nome_preparado}' com '{nome_municipio_api}' - UF: {uf_api}")

        if nome_municipio_api == nome_preparado and uf_api == uf.lower():
            codigo_ibge = municipio['id']
            logger.info(f"Município '{nome_preparado}' encontrado na base IBGE com código {codigo_ibge}.")
            return {
                "municipio_original": nome_municipio,
                "nome_comparacao": nome_preparado,
                "codigo_ibge": codigo_ibge
            }

    # Caso não encontre na base IBGE, tenta no dicionário de ids não mapeados
    if nome_preparado in ids_nao_mapeados:
        codigo_ibge = ids_nao_mapeados[nome_preparado]
        logger.info(f"Município '{nome_preparado}' não encontrado na base IBGE, mas presente no dicionário de não mapeados com código {codigo_ibge}.")
        return {
            "municipio_original": nome_municipio,
            "nome_comparacao": nome_preparado,
            "codigo_ibge": codigo_ibge
        }

    # Se não encontrou em nenhum lugar
    logger.warning(f"Código IBGE não encontrado para {nome_municipio}.")
    return {
        "municipio_original": nome_municipio,
        "nome_comparacao": nome_preparado,
        "codigo_ibge": -1
    }



def solicitar_codigo_manual(nome_municipio: str) -> int:
    while True:
        try:
            codigo_manual = int(input(f"Informe o código IBGE para {nome_municipio}: "))
            ids_nao_mapeados[preparar_nome_municipio(nome_municipio)] = codigo_manual  # Salva para futuras consultas
            return codigo_manual
        except ValueError:
            print("Código inválido. Insira um número válido.")

def formatar_data_brasileira(data_iso: str) -> str:
    """
    Converte a data do formato ISO para o formato brasileiro.
    """
    try:
        return datetime.strptime(data_iso, '%Y-%m-%d').strftime('%d/%m/%Y')
    except ValueError:
        logger.error(f"Erro ao formatar a data: {data_iso}")
        return "Data inválida"

def formatar_temperatura(temperatura: str) -> float:
    """
    Converte a temperatura para um número float.
    """
    if temperatura is None:
        logger.warning("Temperatura não informada (None). Retornando 0.0 como valor padrão.")
        return 0.0
    try:
        return round(float(temperatura), 1)
    except (TypeError, ValueError) as e:
        logger.error(f"Erro ao formatar a temperatura: '{temperatura}'. Detalhes: {e}")
        return 0.0

def calcular_intensidade(temp_min: str) -> str:
    """
    Calcula a intensidade da geada com base na temperatura mínima.
    """
    try:
        temp_min_float = float(temp_min)
        if temp_min_float < 1:
            return "Forte"
        elif temp_min_float <= 3:
            return "Moderada"
        else:
            return "Fraca"
    except (TypeError, ValueError):
        return "N/A"

def buscar_id_por_nome(cidades, nome_cidade: str, uf: str) -> int:
    """
    Busca o código IBGE pelo nome e UF na lista de cidades. 
    Se não encontrar, tenta buscar no dicionário de ids_nao_mapeados.
    """
    nome_normalizado = preparar_nome_municipio(nome_cidade)
    for cidade in cidades:
        nome_cidade_api = preparar_nome_municipio(cidade["nome"])
        uf_api = cidade["microrregiao"]["mesorregiao"]["UF"]["sigla"].lower()
        if nome_cidade_api == nome_normalizado and uf_api == uf.lower():
            return cidade["id"]

    # Fallback para o dicionário de ids não mapeados
    if nome_normalizado in ids_nao_mapeados:
        logger.info(f"Município '{nome_normalizado}' encontrado no dicionário com código {ids_nao_mapeados[nome_normalizado]}")
        return ids_nao_mapeados[nome_normalizado]

    logger.warning(f"Município '{nome_normalizado}' não encontrado. Retornando -1.")
    return -1

def extrair_dados_geada(folder_path: str = r"C:\Users\ana.brum\Desktop\DadosMeteorologicos"):
    """
    Extrai os dados de geadas da API e salva em um arquivo Excel.
    """
    cidades = fazer_requisicao("https://servicodados.ibge.gov.br/api/v1/localidades/municipios")
    if not cidades:
        logger.error("Não foi possível obter a lista de municípios.")
        return

    data_inicio = datetime(2017, 1, 1)
    data_fim = datetime(2024, 9, 30)
    dados_tratados = []

    while data_inicio <= data_fim:
        ano = data_inicio.year
        mes = data_inicio.month
        nome_mes = data_inicio.strftime('%B').capitalize()

        logger.info(f"Extraindo dados para: {nome_mes} de {ano}")

        primeiro_dia = data_inicio.strftime("%Y-%m-%d")
        ultimo_dia = (data_inicio + timedelta(days=calendar.monthrange(ano, mes)[1] - 1)).strftime("%Y-%m-%d")
        url = f"https://apitempo.inmet.gov.br/geada/{primeiro_dia}/{ultimo_dia}/CONVENCIONAL"
        dados = fazer_requisicao(url)

        # Tratamento dos dados
        if dados:
            for item in dados:
                uf = item.get("UF", "N/A")
                nome_cidade = item.get("NOME", "N/A").title()
                nome_cidade_limpo = preparar_nome_municipio(nome_cidade)
                data_ocorrencia = formatar_data_brasileira(item.get("DT_MEDICAO"))
                temp_min = item.get("TEMP_MIN")

                # Verifica se a temperatura é None ou inválida
                if temp_min is None:
                    logger.warning(f"Ignorando registro: Temperatura mínima não informada para {nome_cidade_limpo} em {data_ocorrencia}.")
                    continue

                try:
                    temperatura_formatada = round(float(temp_min), 1)
                except (TypeError, ValueError) as e:
                    logger.warning(f"Ignorando registro: Temperatura inválida '{temp_min}' para {nome_cidade_limpo} em {data_ocorrencia}. Detalhes: {e}")
                    continue

                intensidade = calcular_intensidade(temp_min)
                id_cidade = buscar_id_por_nome(cidades, nome_cidade_limpo, uf)
                item["Estação"] = "Automatica"

                # Adicionando os dados tratados à lista
                dados_tratados.append([id_cidade, uf, nome_cidade, data_ocorrencia, temperatura_formatada, intensidade,Estação])

        # Avançar para o próximo mês
        data_inicio += timedelta(days=calendar.monthrange(ano, mes)[1])

    # Criar DataFrame com os dados tratados
    colunas = ["Cod. IBGE", "Uf", "Município", "Dia de ocorrência", "Temperatura Mínima", "Intensidade", "Estação"]
    df = pd.DataFrame(dados_tratados, columns=colunas)

    # Certificando-se de que a pasta existe
    os.makedirs(folder_path, exist_ok=True)

    # Caminho completo do arquivo Excel
    excel_file = os.path.join(folder_path, "dados_geada_convencional.xlsx")
    df.to_excel(excel_file, index=False)
    logger.info(f"Dados extraídos e salvos com sucesso no arquivo '{excel_file}'.")

if __name__ == "__main__":
    extrair_dados_geada()

19/02/2025 20:58:04 - INFO - Extraindo dados para: January de 2017
19/02/2025 20:58:04 - INFO - Extraindo dados para: February de 2017
19/02/2025 20:58:06 - INFO - Extraindo dados para: March de 2017
19/02/2025 20:58:06 - INFO - Extraindo dados para: April de 2017
19/02/2025 20:58:07 - INFO - Extraindo dados para: May de 2017
19/02/2025 20:58:07 - INFO - Extraindo dados para: June de 2017
19/02/2025 20:58:08 - WARNING - Ignorando registro: Temperatura mínima não informada para Campos Do Jordão em 26/06/2017.
19/02/2025 20:58:08 - INFO - Extraindo dados para: July de 2017
19/02/2025 20:58:08 - WARNING - Ignorando registro: Temperatura mínima não informada para Campos Do Jordão em 05/07/2017.
19/02/2025 20:58:09 - INFO - Extraindo dados para: August de 2017
19/02/2025 20:58:09 - INFO - Extraindo dados para: September de 2017
19/02/2025 20:58:10 - INFO - Extraindo dados para: October de 2017
19/02/2025 20:58:10 - INFO - Extraindo dados para: November de 2017
19/02/2025 20:58:11 - INFO - E